In [ ]:
!which python

In [ ]:
page = wikipedia.page(title="Apple Inc.", auto_suggest=True)
page.content

In [ ]:
import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 

# Load English language model
nlp = spacy.load("en_core_web_sm")

text = "Apple Inc. was founded by Steve in Cupertino, California."
doc = nlp(text)

for token in doc:
    if token.dep_ == "nsubj" and token.head.dep_ == "ROOT":
        print(f"Subject: {token.text}, Action: {token.head.text}")

# Function to extract relationships
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]
# Extract and print relationships
relationships = get_entities(text)
for rel in relationships:
    print(rel)

In [ ]:
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1",[pattern]) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [ ]:
get_relation(text)

In [ ]:
# from transformers import pipeline
# qa = pipeline("question-answering")
# result = qa(question="Who founded Apple?", context="Apple was founded by Steve Jobs in 1976.")
# print(result['answer'])

import spacy
nlp = spacy.load("en_core_web_sm")
text = "Apple was founded by Steve Jobs in 1976 ."
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_)